In [111]:
%matplotlib inline

import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils

plt.rcParams['figure.figsize'] = (17, 5)

In [113]:
# Directory where mp3 are stored.
AUDIO_DIR = os.environ.get('AUDIO_DIR')

# Load metadata and features.
tracks = utils.load('data/fma_metadata/tracks.csv')
genres = utils.load('data/fma_metadata/genres.csv')
#features = utils.load('data/fma_metadata/features.csv')
#echonest = utils.load('data/fma_metadata/echonest.csv')

#tracks_df = pd.read_csv('data/fma_metadata/tracks.csv')

#np.testing.assert_array_equal(features.index, tracks.index)
#assert echonest.index.isin(tracks.index).all()

tracks.shape, genres.shape, features.shape, echonest.shape

((106574, 52), (163, 4), (106574, 518), (13129, 249))

In [114]:
tracks.dtypes

album   comments                      int64
        date_created         datetime64[ns]
        date_released        datetime64[ns]
        engineer                     object
        favorites                     int64
        id                            int64
        information                category
        listens                       int64
        producer                     object
        tags                         object
        title                        object
        tracks                        int64
        type                       category
artist  active_year_begin    datetime64[ns]
        active_year_end      datetime64[ns]
        associated_labels            object
        bio                        category
        comments                      int64
        date_created         datetime64[ns]
        favorites                     int64
        id                            int64
        latitude                    float64
        location                

In [115]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106574 entries, 2 to 155320
Data columns (total 52 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   (album, comments)            106574 non-null  int64         
 1   (album, date_created)        103045 non-null  datetime64[ns]
 2   (album, date_released)       70294 non-null   datetime64[ns]
 3   (album, engineer)            15295 non-null   object        
 4   (album, favorites)           106574 non-null  int64         
 5   (album, id)                  106574 non-null  int64         
 6   (album, information)         83149 non-null   category      
 7   (album, listens)             106574 non-null  int64         
 8   (album, producer)            18060 non-null   object        
 9   (album, tags)                106574 non-null  object        
 10  (album, title)               105549 non-null  object        
 11  (album, tracks)           

In [133]:
#tracks['genres']
#tracks.get('genres')
tracks.columns

MultiIndex([( 'album',          'comments'),
            ( 'album',      'date_created'),
            ( 'album',     'date_released'),
            ( 'album',          'engineer'),
            ( 'album',         'favorites'),
            ( 'album',                'id'),
            ( 'album',       'information'),
            ( 'album',           'listens'),
            ( 'album',          'producer'),
            ( 'album',              'tags'),
            ( 'album',             'title'),
            ( 'album',            'tracks'),
            ( 'album',              'type'),
            ('artist', 'active_year_begin'),
            ('artist',   'active_year_end'),
            ('artist', 'associated_labels'),
            ('artist',               'bio'),
            ('artist',          'comments'),
            ('artist',      'date_created'),
            ('artist',         'favorites'),
            ('artist',                'id'),
            ('artist',          'latitude'),
          

In [116]:
tracks.describe()

album                                              \
            comments      favorites             id       listens   
count  106574.000000  106574.000000  106574.000000  1.065740e+05   
mean        0.394946       1.286927   12826.933914  3.212031e+04   
std         2.268915       3.133035    6290.261805  1.478532e+05   
min        -1.000000      -1.000000      -1.000000 -1.000000e+00   
25%         0.000000       0.000000    7793.000000  3.361000e+03   
50%         0.000000       0.000000   13374.000000  8.982000e+03   
75%         0.000000       1.000000   18203.000000  2.363500e+04   
max        53.000000      61.000000   22940.000000  3.564243e+06   

                             artist                                \
              tracks       comments      favorites             id   
count  106574.000000  106574.000000  106574.000000  106574.000000   
mean       19.721452       1.894702      30.041915   12036.770404   
std        39.943673       6.297679     100.511408    6881.420867   
min        -1.000000      -1.000000      -1.000000       1.000000   
25%         7.000000       0.000000       1.000000    6443.000000   
50%        11.000000       0.000000       5.000000   12029.500000   
75%        17.000000       1.000000      16.000000   18011.000000   
max       652.000000      79.000000     963.000000   24357.000000   

                                           track                 \
           latitude     longitude       bit_rate       comments   
count  44544.000000  44544.000000  106574.000000  106574.000000   
mean      39.901626    -38.668642  263274.695048       0.031621   
std       18.240860     65.237220   67623.443584       0.321993   
min      -45.878760   -157.526855      -1.000000       0.000000   
25%       39.271398    -79.997459  192000.000000       0.000000   
50%       41.387917    -73.554431  299914.000000       0.000000   
75%       48.856930      4.351710  320000.000000       0.000000   
max       67.286005    175.277000  448000.000000      37.000000   

                                                                  \
            duration      favorites      interest        listens   
count  106574.000000  106574.000000  1.065740e+05  106574.000000   
mean      277.849100       3.182521  3.541310e+03    2329.353548   
std       305.518553      13.513820  1.901743e+04    8028.070647   
min         0.000000       0.000000  2.000000e+00       0.000000   
25%       149.000000       0.000000  5.990000e+02     292.000000   
50%       216.000000       1.000000  1.314000e+03     764.000000   
75%       305.000000       3.000000  3.059000e+03    2018.000000   
max     18350.000000    1482.000000  3.293557e+06  543252.000000   

                      
              number  
count  106574.000000  
mean        8.260945  
std        15.243271  
min         0.000000  
25%         2.000000  
50%         5.000000  
75%         9.000000  
max       255.000000

In [88]:
print(genres.head(15))

          #tracks  parent                title  top_level
genre_id                                                 
1            8693      38          Avant-Garde         38
2            5271       0        International          2
3            1752       0                Blues          3
4            4126       0                 Jazz          4
5            4106       0            Classical          5
6             914      38              Novelty         38
7             217      20               Comedy         20
8             868       0  Old-Time / Historic          8
9            1987       0              Country          9
10          13845       0                  Pop         10
11            367      14                Disco         14
12          32923       0                 Rock         12
13            730       0       Easy Listening         13
14           1499       0             Soul-RnB         14
15          34413       0           Electronic         15


In [117]:
print(tracks.head(0))

Empty DataFrame
Columns: [(album, comments), (album, date_created), (album, date_released), (album, engineer), (album, favorites), (album, id), (album, information), (album, listens), (album, producer), (album, tags), (album, title), (album, tracks), (album, type), (artist, active_year_begin), (artist, active_year_end), (artist, associated_labels), (artist, bio), (artist, comments), (artist, date_created), (artist, favorites), (artist, id), (artist, latitude), (artist, location), (artist, longitude), (artist, members), (artist, name), (artist, related_projects), (artist, tags), (artist, website), (artist, wikipedia_page), (set, split), (set, subset), (track, bit_rate), (track, comments), (track, composer), (track, date_created), (track, date_recorded), (track, duration), (track, favorites), (track, genre_top), (track, genres), (track, genres_all), (track, information), (track, interest), (track, language_code), (track, license), (track, listens), (track, lyricist), (track, number), (tr

In [185]:
#tracks['track','genres'] #Working with MultiIndex
#tracks['track']
tracks['track','genre_top']

track_id
2         Hip-Hop
3         Hip-Hop
5         Hip-Hop
10            Pop
20            NaN
           ...   
155316       Rock
155317       Rock
155318       Rock
155319       Rock
155320        NaN
Name: (track, genre_top), Length: 106574, dtype: category
Categories (16, object): ['Blues', 'Classical', 'Country', 'Easy Listening', ..., 'Pop', 'Rock', 'Soul-RnB', 'Spoken']

In [188]:
tracks.keys
#tracks.loc[:,'Electronic'] #want to extract all rows with 'Electronic' in their genre column

<bound method NDFrame.keys of             album                                                              \
         comments        date_created date_released engineer favorites     id   
track_id                                                                        
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4      6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2      4   
...           ...                 ...           ...      ...       ...    ...   
155316          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155317          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155318          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940  

In [36]:
electronic_dict={}
for track in tracks:
    if 'genre_top' == 'Electronic':
        electronic_dict[''] = ''

NameError: name 'track_genre' is not defined

In [195]:
# 1 create new dataframe from electronic genre
# 2 add electronic entries to it

e_df = pd.DataFrame()

for track in tracks:
    if 'genre_top' == 'Electronic':
        e_df = tracks[['track','genre_top']].copy()

In [196]:
print(e_df)

Empty DataFrame
Columns: []
Index: []


In [200]:
for col in tracks:
    print(col)

('album', 'comments')
('album', 'date_created')
('album', 'date_released')
('album', 'engineer')
('album', 'favorites')
('album', 'id')
('album', 'information')
('album', 'listens')
('album', 'producer')
('album', 'tags')
('album', 'title')
('album', 'tracks')
('album', 'type')
('artist', 'active_year_begin')
('artist', 'active_year_end')
('artist', 'associated_labels')
('artist', 'bio')
('artist', 'comments')
('artist', 'date_created')
('artist', 'favorites')
('artist', 'id')
('artist', 'latitude')
('artist', 'location')
('artist', 'longitude')
('artist', 'members')
('artist', 'name')
('artist', 'related_projects')
('artist', 'tags')
('artist', 'website')
('artist', 'wikipedia_page')
('set', 'split')
('set', 'subset')
('track', 'bit_rate')
('track', 'comments')
('track', 'composer')
('track', 'date_created')
('track', 'date_recorded')
('track', 'duration')
('track', 'favorites')
('track', 'genre_top')
('track', 'genres')
('track', 'genres_all')
('track', 'information')
('track', 'inte

In [201]:
print(tracks[('track', 'genre_top')].dtypes)

category
